**Imports**

In [ ]:
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import pickle

**Paper sentences extraction. Save as pickle.**

In [ ]:
covid_papers_df = pd.read_csv('covid_metadata.tsv', sep='\t', header=0)

In [ ]:
covid_papers_df.head()

,Unnamed: 0,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,publish_time_year,date_1
0,5145,45f82c4c5a222af95b1967059617a651f39d87e4,Elsevier,Thrombocytopenia is associated with severe cor...,10.1016/j.cca.2020.03.022,NaN,NaN,els-covid,Abstract Background Coronavirus disease 2019 (...,2020-03-13,"Lippi, Giuseppe; Plebani, Mario; Michael Henry...",Clinica Chimica Acta,NaN,NaN,True,custom_license,2020.0,3
1,7956,849f0d4e93647e40b3a6f0841ebb2dd6a890a6b7,Elsevier,COVID-19 and Italy: what next?,10.1016/S0140-6736(20)30627-9,NaN,NaN,els-covid,Summary The spread of severe acute respiratory...,2020-03-13,"Remuzzi, Andrea; Remuzzi, Giuseppe",The Lancet,NaN,NaN,True,custom_license,2020.0,3
2,8336,4c1d608959033784b3e85e34a9209ba77492f5c6,Elsevier,Clinical and computed tomographic imaging feat...,10.1016/j.jinf.2020.02.017,NaN,NaN,els-covid,Summary Purpose To investigate the clinical an...,2020-02-25,"Xu, Yu-Huan; Dong, Jing-Hui; An, Wei-Min; Lv, ...",Journal of Infection,NaN,NaN,True,custom_license,2020.0,3
3,13438,a2d4f42c48161b18c14ebd3e9a844ecaac536680,Elsevier,Teicoplanin: an alternative drug for the treat...,10.1016/j.ijantimicag.2020.105944,NaN,NaN,els-covid,"Abstract In December 2019, a new coronavirus, ...",2020-03-13,"Baron, Sophie Alexandra; Devaux, Christian; Co...",International Journal of Antimicrobial Agents,NaN,NaN,True,custom_license,2020.0,3
4,16593,9f2a4c05fe00ecbe654be722b6e6997f5232b8d3,Elsevier,"Clinical, laboratory and imaging features of C...",10.1016/j.tmaid.2020.101623,NaN,NaN,els-covid,Abstract Introduction An epidemic of Coronavir...,2020-03-13,"Rodriguez-Morales, Alfonso J.; Cardona-Ospina,...",Travel Medicine and Infectious Disease,NaN,NaN,True,custom_license,2020.0,3


In [ ]:
covid_papers_set = set(covid_papers_df["sha"]) #covid papers id

In [ ]:
display(covid_papers_set)

{'...'}

In [ ]:
str_path_noncomm = 'noncomm_use_subset/noncomm_use_subset/'
str_path_comm = 'comm_use_subset/comm_use_subset/'
str_path_biox = 'biorxiv_medrxiv/biorxiv_medrxiv/'
str_path_custom = 'custom_license/custom_license/'

In [ ]:
papers_covid_dict = {}

In [ ]:
def extract_from_file(file_):
  
  list_list_sentence_word=[]
  doc = json.load(open( file_))
  #print(doc['abstract'])
  paper = {}
  paperId = doc['paper_id']
  if paperId in covid_papers_set:
    title = doc["metadata"]["title"]
    #print(title)
    paper["title"] = title.strip().split(". ")
    abstract = []
    for abstractSection in doc['abstract']:
      textAbstract = abstractSection['text']
      abstract +=  textAbstract.strip().split(". ")
    paper["abstract"] = abstract
    text = []
    for textSection in doc['body_text']:
      textText = textSection["text"]
      text += textText.strip().split(". ")
    paper["text"] = text
    return paperId, paper

  else: 
    return None, None

  



In [ ]:
for folder in [str_path_noncomm,str_path_comm,str_path_biox,str_path_custom]:
    #print(folder)
    list_files = [folder+x for x in os.listdir(folder)]
    for filename in tqdm(list_files):
        paperId, paper = extract_from_file(filename)
        if paperId != None:
          papers_covid_dict[paperId] = paper

In [ ]:
display(len(papers_covid_dict))

453

In [ ]:
display((papers_covid_dict["09c84b9de7fc5c3fed4736e7a3cd45e112377042"]))

{'title': ['COVID-19 in the Shadows of MERS-CoV in the Kingdom of Saudi Arabia'],
 'abstract': [],
 'text': ['Middle East Respiratory Syndrome Coronavirus (MERS-CoV) has plagued the Middle East since it was first reported in 2012 [1] ',
  'As of January 2020, the World Health Organization (WHO) has been notified of 2494 laboratory-confirmed cases with 858 mortalities [2] , with the majority of cases reported from the Kingdom of Saudi Arabia (KSA), which-at the time of this writing-has a total of 2131 laboratory-confirmed cases with 827 deaths [3] , making the case fatality rate (CFR) 39%',
  'Currently, KSA continues to report MERS-CoV cases on weekly basis, but the overall number of nosocomial transmissions has significantly been reduced [4] [5] [6] [7] ',
  'Recently, at the end of December 2019, a cluster of pneumonia cases were reported from Wuhan city, Hubei Province, China, linked to a wet seafood market with a new coronavirus identified as the etiologic agent currently named Sev

In [ ]:
with open("covid_articles.p", "wb+") as embFile:
    pickle.dump(papers_covid_dict, embFile)